# #4 Discovering Butterfree - Streaming Feature Sets

This tutorial will cover how to create a feature set having a streaming data source.

Before diving into the tutorial make sure you have seen the other tutorials, and have a basic understanding of the main features of the library, since this tutorial will not explain it again.


## Example:
Simulating the following scenario:

- We have a streaming JSON data source with events of Pokémons being captured in real time.
- We have a Pokédex data set with more information about specific Pokémons


Objective: 

We want to parse the JSON from the streaming source, get the desired information merging the records with enriched data coming from Pokédex dataset.
Our desire is to have a real time output with the schema:

- **id_capture**: int
- **timestamp**: timestamp
- **id_pokemon**: int
- **pokemon_name**: string
- **pokemon_type**: string

The following code blocks will show how to generate this feature set using Butterfree library:

In [1]:
# setup spark
from pyspark import SparkContext, SparkConf
from pyspark.sql import session

conf = SparkConf().set('spark.driver.host','127.0.0.1')
sc = SparkContext(conf=conf)
spark = session.SparkSession(sc)

In [2]:
# fix working dir
import pathlib
import os

try:
    print(path)
except NameError:
    path = os.path.join(pathlib.Path().absolute(), '../..')

os.chdir(path)
EXAMPLE_PATH = "examples/streaming_feature_set/" 

In [3]:
# butterfree spark client
from butterfree.clients import SparkClient

spark_client = SparkClient()

In [4]:
# stream data generator
from datetime import datetime
import glob
import json
import os
import random
from time import monotonic, sleep
from multiprocessing import Process

IDS = [1, 2, 3, 4, 5, 6]
POKEBALLS = ["Normal", "Great", "Ultra"]


def clean_events_folder():
    files = glob.glob(f"{EXAMPLE_PATH}events/*")
    for f in files:
        os.remove(f)


def create_random_event(counter=0):
    return {
        "id": counter,
        "timestamp": round(monotonic() * 1000),
        "payload": f"{{\"id_pokemon\": {random.choice(IDS)}, \"pokeball\": \"{random.choice(POKEBALLS)}\"}}"
        }

def start_stream(wait_time=2):
    clean_events_folder()
    counter = 0
    while True:
        event = create_random_event(counter)
        file_name = f"{EXAMPLE_PATH}events/{event['timestamp']}.json"
        with open(file_name, "w") as file:
            json.dump(event, file)
        sleep(wait_time)
        counter += 1

p = Process(target=start_stream)
p.start()

### Showing test data

In [5]:
events_df = spark.read.json("events/")
# events_df.createOrReplaceTempView("events")  # creating listing_events table

print(">>> events :")
events_df.toPandas()

>>> events :


,id,payload,timestamp
0,0,"{""id_pokemon"": 6, ""pokeball"": ""Great""}",32922718


In [6]:
print(">>> pokedex.json file:")
spark.read.json("pokedex.json").toPandas()

>>> pokedex.json file:


,id,name,type
0,1,Geodude,Rock
1,2,Bulbasaur,Grass
2,3,Pikachu,Electric
3,4,Eevee,Normal
4,5,Oddish,Grass
5,6,Magikarp,Water


### Extract

- This example shows that dealing with streaming sources with butterfree does not change the code too much.
- We are just going to enable a Stream flag, so using Spark's `readStream` API.
- We are going to use a `pre_processing` named `explode_json_column` to make it easier to access the payload fields.

In [7]:
# Building expected schema from payload

from pyspark.sql.types import IntegerType, StringType, StructField, StructType

payload_schema = StructType(
    [StructField("id_pokemon", IntegerType()), StructField("pokeball", StringType())]
)

In [8]:
from butterfree.extract import Source
from butterfree.extract.readers import FileReader
from butterfree.extract.pre_processing import explode_json_column

readers = [
    FileReader(id="events", path="events/", format="json", stream=True).with_(
        transformer=explode_json_column, column="payload", json_schema=payload_schema
    ),
    FileReader(id="pokedex", path="pokedex.json", format="json")
]

query = """
select
    events.*,
    pokedex.name,
    pokedex.type
from
    events
    join pokedex
        on events.id_pokemon = pokedex.id
"""

source = Source(readers=readers, query=query)

In [9]:
source_df = source.construct(spark_client)

In [10]:
# showing that it is a Spark's streaming df
source_df.isStreaming

True

In [11]:
# schema
source_df.schema.simpleString()

'struct<id:bigint,payload:string,timestamp:bigint,id_pokemon:int,pokeball:string,name:string,type:string>'

### Transform

In [12]:
from butterfree.transform import FeatureSet
from butterfree.transform.features import Feature, KeyFeature, TimestampFeature
from butterfree.transform.transformations import SQLExpressionTransform
from butterfree.transform.transformations import H3HashTransform
from butterfree.constants import DataType

keys = [
    KeyFeature(
        name="id_capture",
        description="Unique identificator code for pokemons.",
        from_column="id",
        dtype=DataType.INTEGER,
    )
]

# from_ms = True because the data originally is not in a Timestamp format.
ts_feature = TimestampFeature(from_ms=True)

features = [
    Feature(
        name="id_pokemon",
        description="Pokemon unique identifier.",
        dtype=DataType.INTEGER,
    ),
    Feature(
        name="pokemon_name",
        description="Pokemon's name.",
        from_column="name",
        dtype=DataType.STRING,
    ),
    Feature(
        name="pokemon_type",
        description="Pokemon's element type.",
        from_column="type",
        dtype=DataType.STRING,
    ),
]

feature_set = FeatureSet(
    name="pokemon_capturing_events",
    entity="events",  # entity: to which "business context" this feature set belongs
    description="Features describring events about Pokemon's capturing and data from Pokedex.",
    keys=keys,
    timestamp=ts_feature,
    features=features,
)

In [13]:
feature_set_df = feature_set.construct(source_df, spark_client)

In [14]:
# showing that it is a Spark's streaming df
feature_set_df.isStreaming

True

In [15]:
# schema
feature_set_df.schema.simpleString()

'struct<id_capture:int,timestamp:timestamp,id_pokemon:int,pokemon_name:string,pokemon_type:string>'

### Load

- Using debug mode with a stream df will create a temporary view with that updates in real time (in memory)

In [16]:
from butterfree.load.writers import (
    OnlineFeatureStoreWriter,
)
from butterfree.load import Sink

writers = [OnlineFeatureStoreWriter(debug_mode=True)]
sink = Sink(writers=writers)

## Pipeline

In [17]:
from butterfree.pipelines import FeatureSetPipeline

pipeline = FeatureSetPipeline(source=source, feature_set=feature_set, sink=sink)

In [18]:
# asinc run when creating an in memory streaming view for sink 
pipeline.run()

### Showing the results

In [19]:
# waiting some time for new records
sleep(10)

In [20]:
print(">>> Online Feature Store pokemon_capturing_events feature set table:")
spark.table("online_feature_store__pokemon_capturing_events").orderBy(
    "id_capture", "timestamp", ascending=False
).toPandas()

>>> Online Feature Store pokemon_capturing_events feature set table:


,id_capture,timestamp,id_pokemon,pokemon_name,pokemon_type
0,8,1970-01-01 06:08:58,5,Oddish,Grass
1,7,1970-01-01 06:08:56,3,Pikachu,Electric
2,6,1970-01-01 06:08:54,3,Pikachu,Electric
3,5,1970-01-01 06:08:52,5,Oddish,Grass
4,4,1970-01-01 06:08:50,1,Geodude,Rock
5,3,1970-01-01 06:08:48,4,Eevee,Normal
6,2,1970-01-01 06:08:46,4,Eevee,Normal
7,1,1970-01-01 06:08:44,5,Oddish,Grass
8,0,1970-01-01 06:08:42,6,Magikarp,Water


In [21]:
# waiting some time for new records
sleep(10)

In [22]:
spark.table("online_feature_store__pokemon_capturing_events").orderBy(
    "id_capture", "timestamp", ascending=False
).toPandas()

,id_capture,timestamp,id_pokemon,pokemon_name,pokemon_type
0,13,1970-01-01 06:09:08,2,Bulbasaur,Grass
1,12,1970-01-01 06:09:06,1,Geodude,Rock
2,11,1970-01-01 06:09:04,5,Oddish,Grass
3,10,1970-01-01 06:09:02,1,Geodude,Rock
4,9,1970-01-01 06:09:00,6,Magikarp,Water
5,8,1970-01-01 06:08:58,5,Oddish,Grass
6,7,1970-01-01 06:08:56,3,Pikachu,Electric
7,6,1970-01-01 06:08:54,3,Pikachu,Electric
8,5,1970-01-01 06:08:52,5,Oddish,Grass
9,4,1970-01-01 06:08:50,1,Geodude,Rock


In [23]:
# waiting some time for new records
sleep(10)

In [24]:
spark.table("online_feature_store__pokemon_capturing_events").orderBy(
    "id_capture", "timestamp", ascending=False
).toPandas()

,id_capture,timestamp,id_pokemon,pokemon_name,pokemon_type
0,18,1970-01-01 06:09:18,3,Pikachu,Electric
1,17,1970-01-01 06:09:16,1,Geodude,Rock
2,16,1970-01-01 06:09:14,1,Geodude,Rock
3,15,1970-01-01 06:09:12,2,Bulbasaur,Grass
4,14,1970-01-01 06:09:10,2,Bulbasaur,Grass
5,13,1970-01-01 06:09:08,2,Bulbasaur,Grass
6,12,1970-01-01 06:09:06,1,Geodude,Rock
7,11,1970-01-01 06:09:04,5,Oddish,Grass
8,10,1970-01-01 06:09:02,1,Geodude,Rock
9,9,1970-01-01 06:09:00,6,Magikarp,Water


- As we can see, the table is being updated in real time, since the data source is being read in stream mode
- We show that we can enrich this events in real-time (Pokédex)

In [25]:
# stop the stream process
p.terminate()